In [14]:
# ----- Core Python & Visualization ----- #
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ----- Data Preprocessing ----- #
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# ----- Models ----- #
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

# ----- Evaluation Metrics ----- #
from sklearn.metrics import r2_score, mean_squared_error, classification_report, confusion_matrix, accuracy_score

# ----- Neural Network (Keras) ----- #
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD

from scikeras.wrappers import KerasClassifier 


In [9]:
import pandas as pd 

df = pd.read_csv("/Users/anhuynh/Downloads/archive (8)/cleansingWine.csv")


/var/folders/v9/hcdjkhvd5nq6llrn61tfl3qr0000gn/T/ipykernel_59743/602357411.py:3: DtypeWarning: Columns (8,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/anhuynh/Downloads/archive (8)/cleansingWine.csv")


In [10]:
# Dataset Overview
print("🧠 Shape of merged data:", df.shape)

# Columns and Data Types
print("\n📋 Column names:\n", df.columns.tolist())
print("\n🔎 Data types:\n", df.dtypes)

# Preview
print("\n🧾 First 5 rows:\n", df.head())

# Missing Data
print("\n🧹 Missing values per column:\n", df.isnull().sum())

# Descriptive Stats for Numeric Columns
print("\n📊 Summary stats (numeric):\n", df.describe())

# Descriptive Stats for Categorical Columns
print("\n🧩 Summary stats (categorical):\n", df.describe(include=['object', 'category']))

# Unique Values Per Column
print("\n🔢 Unique values per column:")
for col in data.columns:
    print(f"  - {col}: {data[col].nunique()} unique values")

# Duplicate Check
print("\n📎 Number of duplicate rows:", df.duplicated().sum())

# Total number of missing values
total_na = df.isna().sum().sum()
print("🧮 Total number of missing values in data:", total_na)

🧠 Shape of merged data: (21605, 32)

📋 Column names:
 ['Unnamed: 0', 'id', 'name', 'producer', 'nation', 'local1', 'local2', 'local3', 'local4', 'varieties1', 'varieties2', 'varieties3', 'varieties4', 'varieties5', 'varieties6', 'varieties7', 'varieties8', 'varieties9', 'varieties10', 'varieties11', 'varieties12', 'type', 'use', 'abv', 'degree', 'sweet', 'acidity', 'body', 'tannin', 'price', 'year', 'ml']

🔎 Data types:
 Unnamed: 0      int64
id              int64
name           object
producer       object
nation         object
local1         object
local2         object
local3         object
local4         object
varieties1     object
varieties2     object
varieties3     object
varieties4     object
varieties5     object
varieties6     object
varieties7     object
varieties8     object
varieties9     object
varieties10    object
varieties11    object
varieties12    object
type           object
use            object
abv            object
degree         object
sweet          object
aci

In [ ]:
# Drop unnecessary columns if they exist
columns_to_drop = ['Unnamed: 0', 'id', 'producer', 'local1', 'local2', 'local3', 'local4', 'name']
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

# Remove rows with missing or non-positive price
df = df[df['price'].notnull() & (df['price'] > 0)]

# Convert sensory columns to numeric by extracting the first digit
sensory_cols = ['sweet', 'acidity', 'body', 'tannin']
for col in sensory_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.extract(r'(\d)').astype(float)

# Convert ranges like "14~15" to average values
def average_range(val):
    try: 
        nums = list(map(float, str(val).split('~')))
        return sum(nums) / len(nums)
    except:
        return np.nan

for col in ['abv', 'degree']:
    if col in df.columns:
        df[col] = df[col].apply(average_range)

# Drop rows with missing critical fields, then fill remaining missing values with 0
df.dropna(subset=['varieties1', 'nation', 'price'], inplace=True)
df.fillna(0, inplace=True)


In [ ]:
# Define target
y = df['nation']
# Drop target from features
X = df.drop(columns=['nation'])

y = pd.get_dummies(y, columns=['nation']) 

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=['varieties1', 'use', 'type', 'varieties2', 'varieties3', 'varieties4', 'varieties5', 'varieties6', 'varieties7', 'varieties8', 'varieties9', 'varieties10', 'varieties11', 'varieties12'])

# Normalize numeric features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer
    Dense(64, activation='relu'),  # Hidden layer
    Dense(y_train.shape[1], activation='softmax')  # Output layer (multiclass classification)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),  # Optimizer with learning rate
              loss='categorical_crossentropy',  # Loss function for multiclass classification
              metrics=['accuracy'])  # Metrics to monitor

# Print the model summary to visualize architecture
model.summary()

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       124,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 28)             │         1,820 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,876 (526.86 KB)

 Trainable params: 134,876 (526.86 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}") 

Epoch 1/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4759 - loss: 1.9303 - val_accuracy: 0.6818 - val_loss: 1.0752
Epoch 2/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7145 - loss: 0.9049 - val_accuracy: 0.6862 - val_loss: 1.0264
Epoch 3/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7328 - loss: 0.8211 - val_accuracy: 0.6972 - val_loss: 1.0130
Epoch 4/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7480 - loss: 0.7682 - val_accuracy: 0.6964 - val_loss: 1.0204
Epoch 5/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7560 - loss: 0.7474 - val_accuracy: 0.7065 - val_loss: 1.0198
Epoch 6/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7565 - loss: 0.7206 - val_accuracy: 0.7057 - val_loss: 1.0405
Epoch 7/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7626 - loss: 0.7244 - val_accuracy: 0.7069 - val_loss: 1.0475
Epoch 8/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7694 - loss: 0.6966 - val_accuracy: 0.